In [1]:
!pip install torch torchvision tqdm ninja click requests
!pip install -r docs/requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [2]:
!pip install opencv-python


In [4]:
# !git clone https://github.com/NVlabs/stylegan3.git


Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.2% (102/163)
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 10.22 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [5]:
# %cd stylegan3


/kaggle/working/stylegan3


In [6]:
#one time intallation as required
!pip install torch torchvision
!pip install numpy scipy pillow requests tqdm pyspng ninja


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 4.4 MB/s eta 0:00:0000:01


In [7]:
!pip install click dnnlib


ERROR: Could not find a version that satisfies the requirement dnnlib (from versions: none)
ERROR: No matching distribution found for dnnlib


In [15]:
#important updation to avoid BETAS and tensor bugs in code
!pip install torch==2.0.1 torchvision==0.15.2


In [11]:
# #MAKING ZIP FILE OF DATASET
# !python dataset_tool.py --source="/kaggle/input/stockdata64x64/niftyimages64x64/Class0" --dest=CLASS0.zip


100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 278.02it/s]


In [16]:
#SHOWING ALL INSIDE KAGGLE WORK
!ls /kaggle/working/*

avg_spectra.py	 Dockerfile	  gui_utils    README.md      visualizer.py
calc_metrics.py  docs		  legacy.py    torch_utils    viz
CLASS0.zip	 environment.yml  LICENSE.txt  training
dataset_tool.py  gen_images.py	  metrics      training-runs
dnnlib		 gen_video.py	  __pycache__  train.py


In [17]:
# #TRAINING FROM SCRATCH
# !python train.py \
#   --outdir=training-runs \
#   --data=CLASS0.zip \
#   --gpus=2 \
#   --batch=4 \
#   --gamma=8.2 \
#   --snap=10 \
#   --cfg=stylegan3-t \
#   --mirror=1 \
#   --aug=ada \
#   --kimg=40 \
#   --cbase=16384 --cmax=64 \
#   --mbstd-group=2 \
#   --metrics=none



Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 16384,
    "channel_max": 64,
    "magnitude_ema_beta": 0.9998613801725043
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 2
    },
    "channel_base": 16384,
    "channel_max": 64
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 8.2
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_factor

In [25]:
#TRAINING DCGAN CLASSWISE

import os
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import shutil

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
z_dim = 100
image_size = 64
batch_size = 128
num_epochs = 100
lr = 0.0002
beta1 = 0.5
data_path = "/kaggle/input/stockdata64x64/niftyimages64x64"

# Transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Generator & Discriminator classes (use your original definition)
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, channels_img, 4, 2, 1),
            nn.Tanh()
        )
    def forward(self, x): return self.main(x)

class Discriminator(nn.Module):
    def __init__(self, channels_img):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(channels_img, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, 4, 1, 0),
            nn.Sigmoid()
        )
    def forward(self, x): return self.main(x)

# Loop through each class folder
class_folders = sorted(os.listdir(data_path))
for class_name in class_folders:
    class_dir = os.path.join(data_path, class_name)
    dataset = ImageFolder(root=data_path, transform=transform)
    dataset.samples = [(os.path.join(class_dir, f), 0) for f in os.listdir(class_dir) if f.endswith(('.jpg', '.png'))]
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Model init
    netG = Generator(z_dim, 1).to(device)
    netD = Discriminator(1).to(device)
    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

    fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)
    save_dir = f"/kaggle/working/generated_images_{class_name}"
    os.makedirs(save_dir, exist_ok=True)

    print(f"\n🔧 Training DCGAN for class: {class_name}")
    for epoch in range(num_epochs):
        for i, (real, _) in enumerate(dataloader):
            real = real.to(device)
            b_size = real.size(0)
            noise = torch.randn(b_size, z_dim, 1, 1, device=device)
            fake = netG(noise)

            label_real = torch.ones(b_size, device=device)
            label_fake = torch.zeros(b_size, device=device)

            # Train Discriminator
            netD.zero_grad()
            lossD = criterion(netD(real).view(-1), label_real) + criterion(netD(fake.detach()).view(-1), label_fake)
            lossD.backward()
            optimizerD.step()

            # Train Generator
            netG.zero_grad()
            lossG = criterion(netD(fake).view(-1), label_real)
            lossG.backward()
            optimizerG.step()

        print(f"[{class_name}] Epoch {epoch+1}/{num_epochs} | Loss D: {lossD.item():.4f} | Loss G: {lossG.item():.4f}")
        with torch.no_grad():
            fake_imgs = netG(fixed_noise).detach().cpu()
            save_image(fake_imgs, f"{save_dir}/fake_epoch_{epoch+1:03}.png", normalize=True)

    # Zip per class
    shutil.make_archive(f"/kaggle/working/{class_name}_images", 'zip', save_dir)



🔧 Training DCGAN for class: Class0
[Class0] Epoch 1/100 | Loss D: 1.5946 | Loss G: 2.6790
[Class0] Epoch 2/100 | Loss D: 0.6538 | Loss G: 4.3511
[Class0] Epoch 3/100 | Loss D: 0.2975 | Loss G: 4.6833
[Class0] Epoch 4/100 | Loss D: 0.1580 | Loss G: 4.9034
[Class0] Epoch 5/100 | Loss D: 0.0851 | Loss G: 5.0318
[Class0] Epoch 6/100 | Loss D: 0.0850 | Loss G: 5.2286
[Class0] Epoch 7/100 | Loss D: 0.0506 | Loss G: 5.5185
[Class0] Epoch 8/100 | Loss D: 0.0494 | Loss G: 5.4103
[Class0] Epoch 9/100 | Loss D: 0.0385 | Loss G: 5.5586
[Class0] Epoch 10/100 | Loss D: 0.0609 | Loss G: 5.9004
[Class0] Epoch 11/100 | Loss D: 0.0262 | Loss G: 6.0247
[Class0] Epoch 12/100 | Loss D: 0.0286 | Loss G: 5.6594
[Class0] Epoch 13/100 | Loss D: 0.0368 | Loss G: 5.9047
[Class0] Epoch 14/100 | Loss D: 0.0240 | Loss G: 6.1390
[Class0] Epoch 15/100 | Loss D: 0.0192 | Loss G: 6.1285
[Class0] Epoch 16/100 | Loss D: 0.0201 | Loss G: 6.0756
[Class0] Epoch 17/100 | Loss D: 0.0214 | Loss G: 6.1450
[Class0] Epoch 18/100

In [26]:
#Generating Images

import torch
from torchvision.utils import save_image
import os
import shutil

# Configuration
z_dim = 100
num_images = 2000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📍 Define your Generator class again if in a separate notebook/cell
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, channels_img, 4, 2, 1),
            nn.Tanh()
        )
    def forward(self, x):
        return self.main(x)

# List your class names
class_names = ['class0', 'class1', 'class2']  # replace with your actual class folder names

for class_name in class_names:
    print(f"🚀 Generating images for class: {class_name}")

    # Initialize generator
    netG = Generator(z_dim, 1).to(device)

    # Load weights if saved (optional, use if you saved model earlier)
    # netG.load_state_dict(torch.load(f'/kaggle/working/generator_{class_name}.pth'))

    # Path to save images
    save_path = f"/kaggle/working/generated_{class_name}"
    os.makedirs(save_path, exist_ok=True)

    with torch.no_grad():
        for i in range(num_images):
            noise = torch.randn(1, z_dim, 1, 1, device=device)
            fake_img = netG(noise)
            save_image(fake_img, os.path.join(save_path, f"{class_name}_{i+1:04}.png"), normalize=True)

    # Zip the folder
    shutil.make_archive(f"/kaggle/working/{class_name}_2000images", 'zip', save_path)
    print(f"✅ {num_images} images saved and zipped for {class_name}")


🚀 Generating images for class: class0
✅ 2000 images saved and zipped for class0
🚀 Generating images for class: class1
✅ 2000 images saved and zipped for class1
🚀 Generating images for class: class2
✅ 2000 images saved and zipped for class2
